In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

### Функция прогнозирования прироста в терминалах на следующий день

In [2]:
def predict_tomorrow(df_full,day):
    df = df_full.drop(['TID'],axis=1)
    days_of_week = [0,1,2,3,4,5,6]
    
    # формируем датафрейм с тренировочной выборкой по ретроспекту
    df2 = pd.DataFrame()
    df2['tid'] = np.repeat(df_full['TID'].values,len(df.columns))
    df2['date'] = np.concatenate(np.repeat([0,df.columns.values],[1,len(df_full['TID'].values)])[1:])
    df2['y'] = np.concatenate(df.values)
    df2['date'] = df2['date'].astype('datetime64')
    df2['weekday'] = df2['date'].dt.dayofweek
    df2['day'] = df2['date'].dt.day

    while days_of_week[0] != df2.iloc[0]['weekday']:
        days_of_week.append(days_of_week[0])
        days_of_week.pop(0)
    
    df_param = pd.DataFrame()
    df_param['tid'] = df_full["TID"].values
    temp = []
    for tid in df_full["TID"].values:
        temp.append(df2[df2['tid'] == tid]['y'].median())
    df_param['median'] = temp
    temp = []
    for tid in df_full["TID"].values:
        temp.append(df2[df2['tid'] == tid]['y'].mean())
    df_param['mean'] = temp

    temp = df2.groupby(by=['tid','weekday']).mean()
    mean_weekday = np.zeros(df2.shape[0])
    for i,j in zip(enumerate(df2['tid'].values),df2['weekday'].values):
        mean_weekday[i[0]] = temp.loc[i[1],j]['y']
        
    temp = df2.groupby(by=['tid','weekday']).median()
    median_weekday = np.zeros(df2.shape[0])
    for i,j in zip(enumerate(df2['tid'].values),df2['weekday'].values):
        median_weekday[i[0]] = temp.loc[i[1],j]['y']

    df2['median'] = np.repeat(df_param['median'].values,len(df.columns))
    df2['mean'] = np.repeat(df_param['mean'].values,len(df.columns))
    df2['mean_weekday'] = mean_weekday
    df2['median_weekday'] = median_weekday
    df2['y(t-1)'] = np.concatenate(df.shift(1, axis="columns").values)
    df2 = df2.dropna()
    
    X_train = df2.drop(['date','y','day','weekday'], axis=1)
    y_train = df2['y']
    
    # формируем датафрейм с тестовой выборкой
    df3 = pd.DataFrame(columns = df2.columns, index=np.arange(0,df_full['TID'].values.shape[0]))
    df3 = df3.drop('y',axis=1)
    df3['tid'] = df_full['TID'].values
    df3['date'] = day
    df3['date'] = df3['date'].astype('datetime64')
    df3['weekday'] = df3['date'].dt.dayofweek
    df3['day'] = df3['date'].dt.day
    df3['median'] = df2.groupby(by=['tid']).mean()['median'].values
    df3['mean'] = df2.groupby(by=['tid']).mean()['mean'].values
    df3['median_weekday'] = df2[df2['weekday'] == df3['weekday'].unique()[0]].groupby(by=['tid']).mean()['median_weekday'].values
    df3['mean_weekday'] = df2[df2['weekday'] == df3['weekday'].unique()[0]].groupby(by=['tid']).mean()['mean_weekday'].values
    df3['y(t-1)'] = df2[df2['date'] == df.columns.values[-1]]['y'].values
    
    X_test = df3.drop(['date','day','weekday'], axis=1)
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    prediction = lr.predict(X_test)
    
    # проверка на отрицательные значения
    prediction[prediction < 0] = 0.0
    
    return pd.Series(prediction)

### Класс алгоритма муравьиной колонии для решения задачи коммивояжёра

In [3]:
class ACO_TSP:
    def __init__(self, func, n_dim, size_pop=10, max_iter=20, distance_matrix=None, alpha=1, beta=2, rho=0.1, car_in=10.0):
        self.func = func
        self.n_dim = n_dim  # количество городов
        self.size_pop = size_pop  # количество муравьёв
        self.max_iter = max_iter  # количество итераций
        self.alpha = alpha  # коэффициент важности феромонов в выборе пути
        self.beta = beta  # коэффициент значимости расстояния
        self.rho = rho  # скорость испарения феромонов
        self.distance_matrix = distance_matrix
        self.prob_matrix_distance = 1 / (distance_matrix + 1e-10 * np.eye(n_dim, n_dim))

        # Матрица феромонов, обновляющаяся каждую итерацию
        self.Tau = np.ones((n_dim, n_dim))
        # Путь каждого муравья в определённом поколении
        self.Table = np.zeros((size_pop, n_dim)).astype(int)
        self.y = None  # Общее расстояние пути муравья в определённом поколении
        self.generation_best_X, self.generation_best_Y = [], [] # фиксирование лучших поколений
        self.x_best_history, self.y_best_history = self.generation_best_X, self.generation_best_Y
        self.best_x, self.best_y = None, None

    def run(self, max_iter=None):
        self.max_iter = max_iter or self.max_iter
        for i in range(self.max_iter):
            # вероятность перехода без нормализации
            prob_matrix = (self.Tau ** self.alpha) * (self.prob_matrix_distance) ** self.beta
            for j in range(self.size_pop):  # для каждого муравья
                # точка начала пути (она может быть случайной, это не имеет значения)
                self.Table[j, 0] = 0
                for k in range(self.n_dim - 1):  # каждая вершина, которую проходят муравьи
                    # точка, которая была пройдена и не может быть пройдена повторно
                    taboo_set = set(self.Table[j, :k + 1])
                    # список разрешённых вершин, из которых будет происходить выбор
                    allow_list = list(set(range(self.n_dim)) - taboo_set)
                    prob = prob_matrix[self.Table[j, k], allow_list]
                    prob[prob==np.inf] = prob[prob!=np.inf].sum() # чтобы не было NaN из-за inf
                    prob = prob / prob.sum() # нормализация вероятности
                    next_point = np.random.choice(allow_list, size=1, p=prob)[0]
                    self.Table[j, k + 1] = next_point

            # рассчёт расстояния
            y = np.array([self.func(i,car_in,self.distance_matrix) for i in self.Table])

            # фиксация лучшего решения
            index_best = y.argmin()
            x_best, y_best = self.Table[index_best, :].copy(), y[index_best].copy()
            self.generation_best_X.append(x_best)
            self.generation_best_Y.append(y_best)

            # подсчёт феромона, который будет добавлен к ребру
            delta_tau = np.zeros((self.n_dim, self.n_dim))
            for j in range(self.size_pop):  # для каждого муравья
                for k in range(self.n_dim - 1):  # для каждой вершины
                    # муравьи перебираются из вершины n1 в вершину n2
                    n1, n2 = self.Table[j, k], self.Table[j, k + 1]
                    delta_tau[n1, n2] += 1 / y[j]  # нанесение феромона
                # муравьи ползут от последней вершины обратно к первой
                n1, n2 = self.Table[j, self.n_dim - 1], self.Table[j, 0]
                delta_tau[n1, n2] += 1 / y[j]  # нанесение феромона

            self.Tau = (1 - self.rho) * self.Tau + delta_tau

        best_generation = np.array(self.generation_best_Y).argmin()
        self.best_x = self.generation_best_X[best_generation]
        self.best_y = self.generation_best_Y[best_generation]
        return self.best_x, self.best_y

    fit = run

### Функции создания маршрута и вычисления длины пути с помощью алгоритма ACO

In [4]:
# вычисление длины пути
def cal_total_distance(routine,car_in,distance_matrix):
    num_points, = routine.shape
    sum_road = car_in # обработка первого терминала 
    for i in range(num_points):
        if (i + 1) % num_points != 0:
            sum_road +=distance_matrix[routine[i % num_points],routine[(i + 1) % num_points]]+car_in
    return sum_road

# создание маршрута
def make_route(distance_matrix,points_coordinate,car_in):
    num_points = len(distance_matrix[0]) # количество вершин
    # создание объекта алгоритма муравьиной колонии
    aca = ACO_TSP(func=cal_total_distance, n_dim=num_points,
                  size_pop=25,  # количество муравьёв
                  max_iter=30, distance_matrix=distance_matrix,car_in=car_in)
    best_x, best_y = aca.run()
    best_points_ = best_x
    best_points_coordinate = points_coordinate[best_points_,:]
    
    # Вывод результатов на экран
#     fig, ax = plt.subplots(1, 2)
#     for index in range(0, len(best_points_)):
#         ax[0].annotate(best_points_[index], (best_points_coordinate[index, 0], best_points_coordinate[index, 1]))
#     ax[0].plot(best_points_coordinate[:, 0],
#                best_points_coordinate[:, 1], 'o-r')
#     pd.DataFrame(aca.y_best_history).cummin().plot(ax=ax[1])
    # изменение размера графиков
#     plt.rcParams['figure.figsize'] = [20, 10]
#     plt.show()
    
    # подсчет времени прибытия в каждый терминал
    dist = [0.0]
    for index in range(0, len(best_points_)-1):
        dist.append(car_in+dist[-1]+distance_matrix[best_points_[index],best_points_[index+1]])
    return best_points_,best_y,dist

### Функция моделирования маршрута на день и подсчета издержек

In [18]:
def model_route(filepath_from = '',filepath_to = '',perc_not_visit = 0.02/365, perc_visit = 0.01/100, min_limit_visit = 100.0,
               max_limit_in_term = 1000000.0, max_limit_days = 14, car_cost = 20000.0, 
               car_start = 8, car_end = 20, car_in = 10, N_cars = 9):

    # Считываем данные по текущему наполненнию терминалов и приростам в ретроспекте 
    df_full = pd.read_csv(filepath_from+'incomes.csv', sep=';', encoding='windows-1251')
    # Считываем данные по времени в пути между терминалами
    df_times = pd.read_csv(filepath_from+'times v4.csv', sep=',', encoding='windows-1251')
    # Считываем данные по координатам терминалов
    df_tids = pd.read_csv(filepath_from+'tids.csv', sep=';', encoding='windows-1251')
    # Считываем данные по датам последних посещений терминалов
    df_days_last_visited = pd.read_csv(filepath_from+'days_last_visited.csv', sep=';', encoding='windows-1251')

    df_sum = pd.DataFrame()
    df_sum['tid'] = df_full['TID']
    # Считываем дни последних посещений терминалов и получаем количество дней с последнего посещения
    df_sum['days_not_visited'] = df_days_last_visited['date'].astype('datetime64')
    
    # дата составления маршрута - сегодня
    today = datetime.datetime.now()
    day = today - datetime.timedelta(hours = today.hour,minutes=today.minute,seconds=today.second,microseconds=today.microsecond)
    df_sum['days_not_visited'] = ((today - df_sum['days_not_visited']) / np.timedelta64 ( 1 , 'D')).astype(int)
    
    # Записываем текущие остатки
    df_sum['counts(t-1)'] = df_full['текущий остаток']

    # датафреймы для отчетов за день 
    log_df_route = []
    log_df_fond = pd.DataFrame()
    log_df_fond['tid'] = df_full['TID']
    log_df_ink = pd.DataFrame()
    log_df_ink['tid'] = df_full['TID']

    # прогнозируем вероятный прирост на завтра
    df_sum['plus_on_day'] = predict_tomorrow(df_full.drop(['текущий остаток'],axis=1),day)
    # сумма остатка и прироста, ожидаемого завтра
    df_sum['counts_to_range'] = df_sum['counts(t-1)'] + df_sum['plus_on_day'] 
    # меняем на 0, если сегодня забрать нечего
    df_sum.loc[df_sum['counts(t-1)'] == 0,'counts_to_range'] = 0
    # прибавляем день (если не посетим сегодня)
    df_sum['days_not_visited'] = df_sum['days_not_visited'] + 1

    # вводим признак обязательных к посещению терминалов по ограничениям задачи
    df_sum['very_need_to_visit'] = 0 
    df_sum.loc[(df_sum['days_not_visited'] > max_limit_days) | (df_sum['counts(t-1)'] >= max_limit_in_term),'very_need_to_visit'] = 3

    # нормализуем остатки и дни
    df_sum['volume_permax'] = df_sum['counts_to_range']/np.max(df_sum[df_sum['very_need_to_visit'] == 0]['counts_to_range'])
    df_sum['days_in_perc'] = df_sum['days_not_visited']/7
    df_sum.loc[df_sum['counts(t-1)'] == 0,'days_in_perc'] = 0

    # считаем коэффициент, отражающий приоритетность посещения терминалов
    df_sum['need_to_visit'] = df_sum['volume_permax'] + df_sum['days_in_perc'] + df_sum['very_need_to_visit']

    # производим отбор терминалов, которые нужно посетить сегодня по уровню их важности на текущий момент
    df_sum = df_sum.sort_values(by=['need_to_visit'],ascending=False, axis=0)

    T_inday = 155 # базовое число терминалов в день
    k_shift=[0,0] # [добавили, уменьшили]
    # подбираем число терминалов для оптимального маршрута
    while True:
        tid_to_visit = df_sum.iloc[:T_inday]['tid'].values

        # Формируем датафрейм, из которого получим матрицу расстояний по всем терминалам, которые посетим сегодня
        df_times2 = df_times[(df_times['Origin_tid'].isin(tid_to_visit)) & df_times['Destination_tid'].isin(tid_to_visit)]
        distance_matrix = np.zeros((df_times2['Destination_tid'].unique().shape[0],df_times2['Destination_tid'].unique().shape[0]))    
        df_times_array = df_times2.values
        for i in range(len(distance_matrix)):
            for j in range(len(distance_matrix)):
                if not i == j:
                    distance_matrix[i][j] = df_times_array[(df_times_array[:,0]==tid_to_visit[i]) & (df_times_array[:,1]==tid_to_visit[j])][:,2][0]
        df_times3 = pd.DataFrame(distance_matrix, columns = tid_to_visit, index = tid_to_visit)

        # формируем датафрейм координат терминалов, которые посетим сегодня
        df_tids2 = pd.DataFrame()
        df_tids2['TID'] = df_times2['Destination_tid'].unique()
        df_tids2 = df_tids2.join(df_tids.set_index('TID'), on='TID')

        # строим единый маршрут
        points_coordinate = df_tids2[['longitude', 'latitude']].values.astype(float)
        while True: # Алгоритм эвристический, иногда не сходится, это редкое явление, но пересчитываем если вдруг так
            try:
                road,PathLength,dist = make_route(distance_matrix,points_coordinate,car_in)
                break
            except:
                continue

        # проверяем длительность маршрута, если она превышает или меньше допустимой, меняем количество точек
        if PathLength < (N_cars*(car_end-car_start)*60-car_in):
            delta_T = np.ceil(((N_cars*(car_end-car_start)*60)-PathLength)/((PathLength-T_inday*car_in)/(T_inday-1)+car_in))
            T_inday = int(T_inday + delta_T)
#             print(PathLength,'меньше, чем надо. добавили ',delta_T)            
            k_shift[0]+=1
            if ((k_shift[0] > 3) & (k_shift[1] > 3)):
                break
            else:
                continue
        elif PathLength > (N_cars*(car_end-car_start)*60):
            delta_T = np.ceil(((N_cars*(car_end-car_start)*60)-PathLength)/((PathLength-T_inday*car_in)/(T_inday-1)+car_in))
            T_inday = int(T_inday + delta_T)
#             print(PathLength,'превысили. убрали ',delta_T)
            k_shift[1]+=1
            continue
        else:
            break

    # сортируем терминалы в порядке по умолчанию для подсчета издержек 
    df_sum = df_sum.sort_index(ascending=True)

    # сохранили % за инкассацию терминалов за день в отчет
    log_df_ink[day] = df_sum['counts(t-1)'].values*perc_visit*df_sum['tid'].isin(tid_to_visit).map({False:0,True:1})
    # учитываем минимальную сумму за инкассирование
    log_df_ink.loc[(log_df_ink[day] < min_limit_visit) & (log_df_ink[day]!=0.0),day] = min_limit_visit 

    df_sum.loc[df_sum['tid'].isin(tid_to_visit),'days_not_visited'] = 0 # обнуляем дни для тех, которые посетили
    df_sum.loc[df_sum['tid'].isin(tid_to_visit),'counts(t-1)'] = 0 # обнуляем хранилище для тех, которые посетили

    log_df_fond[day] = df_sum['counts(t-1)']*perc_not_visit # сохранили сумму фондирования терминалов за день в отчет

    # формируем маршруты объезда терминалов на каждый броневик
    mas_times = [[] for i in range(N_cars)]
    mas_tids = [[] for i in range(N_cars)]
    full_path_times = np.array(dist)
    full_path_tids = road.copy()
    for car in range(N_cars):
        full_path_times= full_path_times-full_path_times[0]
        mas_tids[car] = full_path_tids[full_path_times<=(car_end-car_start)*60-car_in]
        mas_times[car] = full_path_times[full_path_times<=(car_end-car_start)*60-car_in]
        full_path_tids = full_path_tids[full_path_times>(car_end-car_start)*60-car_in]
        full_path_times = full_path_times[full_path_times>(car_end-car_start)*60-car_in]

    # записываем маршруты для машин с временем прибытия и убытия для каждого терминала
    cur_day = day+datetime.timedelta(hours=car_start)
    for j in range(len(mas_tids)):
        for i in range(len(mas_tids[j])):
            cur_time_in = cur_day+datetime.timedelta(minutes=np.ceil(mas_times[j][i]))
            cur_time_out = cur_time_in + datetime.timedelta(minutes=car_in)
            log_df_route.append([j+1, tid_to_visit[mas_tids[j][i]], cur_time_in.isoformat(' '), cur_time_out.isoformat(' ')])

    # сохраняем файлы с маршрутами и издержками за день
    log_df_fond.to_csv(filepath_to+'log_fond.csv', sep=';')
    log_df_ink.to_csv(filepath_to+'log_ink.csv', sep=';')
    log_df_route = pd.DataFrame(log_df_route)
    log_df_route.to_csv(filepath_to+'log_route.csv', sep=';',index=False,header=['№ броневика','Устройство','Время прибытия','Время отъезда'], encoding='windows-1251')
    
    # заменям даты последних посещений для терминалов, которые обработали сегодня и сохраняем в файл
    df_days_last_visited.loc[df_days_last_visited['tid'].isin(tid_to_visit),'date'] = datetime.datetime.date(day)
    df_days_last_visited.to_csv(filepath_from+'days_last_visited.csv', sep=';',columns=['tid','date'],index=False)
    
    # считаем суммарные издержки за день
    sum_all = log_df_ink[day].sum() + log_df_fond[day].sum() + N_cars*car_cost
    print('Суммарные издержки за день: ',round(sum_all,2))

### Пути к файлам, параметры для моделирования и ограничения
### Запуск моделирования маршрута на день

In [19]:
# локальный путь к файлам incomes.csv, times v4.csv, tids.csv, days_last_visited.csv
# если пустой, то файлы находятся в той же папке, откуда запускается модель
filepath_from = '..\\data\\hackathon\\'
# локальный путь для сохранения файлов с маршрутами на день и издержками по терминалам за день
# если пустой, то файлы сохранятся в ту же папку, откуда запускается модель
filepath_to = '..\\data\\hackathon\\'
# параметры для моделирования и ограничения
# величина % в годовых, которую банк платит за неинкассированную сумму денег в терминале
perc_not_visit = 0.02/365
# стоимость обслуживания одного терминала в случае его инкассирования
perc_visit = 0.01/100
# минимальная сумма за инкассирование
min_limit_visit = 100.0
# максимально допустимая сумма денег в терминале
max_limit_in_term = 1000000.0
# максимально допустимое время, в течение которого терминал можно не обслуживать
max_limit_days = 14
# стоимость одного броневика на день
car_cost = 20000.0
# начало и конец рабочего дня броневиков
car_start = 8
car_end = 20
# время простоя броневика на точке (время на инкассирование)
car_in = 10
# число машин в парке
N_cars = 9

# запуск моделирования маршрута
model_route(filepath_from,filepath_to,perc_not_visit,perc_visit,min_limit_visit,max_limit_in_term,max_limit_days,
            car_cost,car_start,car_end,car_in,N_cars)

Суммарные издержки за день:  220675.56
